In [13]:
import os
os.environ['picaso_refdata'] = '/data/pt426/picaso/reference' #THIS MUST GO BEFORE YOUR IMPORT STATEMENT
os.environ['PYSYN_CDBS'] = '/data/pt426/synphot3/grp/redcat/trds' #this is for the stellar data discussed below.

In [14]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import picaso.justplotit as jpi
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
#1 ck tables from roxana
mh = '+000' #log metallicity
CtoO = '100'# CtoO ratio relative to solar

ck_db = f'/data/pt426/kcoeff/sonora_2020_feh{mh}_co_{CtoO}.data.196'
ck = '/data/pt426/kcoeff/sonora_2020_feh+100_co_100.data.196'

print(ck_db)

/data/pt426/kcoeff/sonora_2020_feh+000_co_100.data.196


In [28]:
cl_run = jdi.inputs(calculation="planet", climate = True) # start a calculation

#note you need to put the climate keyword to be True in order to do so
# now you need to add these parameters to your calculation


tint= 2000 # Intrinsic Temperature of your Planet in K (set as a high internal temp due to impact residual heat)
grav = 9.34 # Gravity of your Planet in m/s/s

cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)')) # input gravity
cl_run.effective_temp(tint) # input effective temperature

In [31]:
# Notice The keyword ck is set to True because you want to use the correlated-k opacities for your calculation
# and not the line by line opacities
opacity_ck = jdi.opannection(ck_db=ck) # grab your opacities

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# set to the sun

T_star = 5700 # K, star effective temperature
logg = 4.4 #logg , cgs
metal = 0 # metallicity of star
r_star = 1 # solar radius
semi_major = 1 # star planet distance, AU

cl_run.star(opacity_ck, temp =T_star,metal =metal, logg =logg, radius = r_star,
            radius_unit=u.R_sun,semi_major= semi_major , semi_major_unit = u.AU)#opacity db, pysynphot database, temp, metallicity, logg


In [ ]:
nlevel = 91 # number of plane-parallel levels in your code

#Lets set the max and min at 1e-4 bars and 500 bars
Teq = 300 # planet equilibrium temperature (set to same order of mag as Earth)
pt = cl_run.guillot_pt(Teq, nlevel=nlevel, T_int = tint, p_bottom=2, p_top=-6)
temp_guess = pt['temperature'].values
pressure = pt['pressure'].values

In [ ]:
nofczns = 1 # number of convective zones initially. Let's not play with this for now.

nstr_upper = 85 # top most level of guessed convective zone
nstr_deep = nlevel -2 # this is always the case. Dont change this
nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones

# Here are some other parameters needed for the code.
rfacv = 0.5 #we are focused on a brown dwarf so let's keep this as is

In [ ]:
cl_run.inputs_climate(temp_guess=temp_guess, pressure=pressure,
                      nstr=nstr, nofczns=nofczns , rfacv=rfacv)

In [ ]:
out = cl_run.climate(opacity_ck, save_all_profiles=True,with_spec=True)